In [ ]:
!pip3 install pm4py pandas

In [7]:
# Import CSV files and drop Inicio column
# Will be using Fin as timestamp
import pandas as pd

df_csv = pd.read_csv(
  "./files/log_gestionar_alarma_1.csv", sep=";", encoding="latin-1",
).drop(columns=["Inicio"])
df_csv.head()

,ID caso,Nombre Actividad,Fin,Ejecutor
0,1,Alarma domiciliaria,01-06-2021 07:19:56,Sistema
1,1,Revisar cámaras,01-06-2021 07:23:46,Bastian
2,1,Te atrapé,01-06-2021 07:23:46,Bastian
3,1,Señal persona casa,01-06-2021 07:25:29,Bastian
4,2,Alarma domiciliaria,01-06-2021 07:53:58,Sistema


### Convert the received csv to a `event log` object

In [ ]:
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

df_log_titles = pm4py.format_dataframe(df_csv, case_id="ID caso", activity_key="Nombre Actividad", timestamp_key="Fin")
events_log = log_converter.apply(df_log_titles, variant=log_converter.Variants.TO_EVENT_LOG)
events_log

# Exploratory Analysis

### Get the number of cases

In [ ]:
len(events_log)

### Get Event logs as Data frames

In [ ]:
df_events_log = log_converter.apply(
    events_log,
    variant=log_converter.Variants.TO_DATA_FRAME
)
df_events_log

### Frequency of activities

In [ ]:
df_events_log["Nombre Actividad"].value_counts()

### Starting and Ending activities

In [ ]:
from pm4py.algo.filtering.log.start_activities import start_activities_filter
from pm4py.algo.filtering.log.end_activities import end_activities_filter

start_activities = start_activities_filter.get_start_activities(events_log)
print("Starting Activities:")
print(start_activities)

end_activities = end_activities_filter.get_end_activities(events_log)
print("Ending Activities:")
print(end_activities)

### Existing variants

In [ ]:
from pm4py.algo.filtering.log.variants import variants_filter

variants = variants_filter.get_variants(events_log)
len(variants)

### Executor frequency

In [ ]:
df_events_log["Ejecutor"].value_counts()

### Median/Mean of the duration of the process

In [ ]:
from pm4py.statistics.traces.generic.log import case_statistics

all_case_durations = case_statistics.get_all_case_durations(
    events_log,
    parameters={
        case_statistics.Parameters.TIMESTAMP_KEY: "time:timestamp"
    },
)

import numpy as np

print(f"Median: {np.median(all_case_durations)}")
print(f"Mean: {np.mean(all_case_durations)}")
print(f"Standard Deviation: {np.std(all_case_durations)}")

# Process Discovery

### All the posible behaviour

In [ ]:
petri_net, initial_marking, final_marking = pm4py.discover_petri_net_alpha(events_log)
pm4py.view_petri_net(petri_net, initial_marking, final_marking, format='png')

In [ ]:
pm4py.save_vis_petri_net(petri_net, initial_marking, final_marking, file_path="./files/petrinet_alpha_events_log.png")